# Train reward model with human feedback

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=802916929536, available=780885438464, percent=2.7, used=17583292416, free=745326710784, active=20479639552, inactive=34074345472, buffers=0, cached=40006926336, shared=17158144, slab=1288433664)


In [3]:
%pip install --disable-pip-version-check -q \
    transformers==4.26.1 \
    datasets==2.9.0 \
    accelerate==0.17.0 \
    bitsandbytes==0.37.0 \
    promptsource==0.2.3 \
    trl==0.4.1 \
    evaluate==0.4.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [5]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client("sagemaker", region)
a2i = boto3.client("sagemaker-a2i-runtime")
s3 = boto3.client("s3", region)

In [6]:
import os
import glob
import numpy as np
import argparse
import pprint
from collections import defaultdict

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader

from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Retrieve the `human_loops_started`

In [7]:
%store -r human_loops_started

In [8]:
try:
    human_loops_started
except NameError:
    print("*** PLEASE RUN PREVIOUS NOTEBOOK BEFORE CONTINUING ***")

In [9]:
print(human_loops_started)

['b30ce452-0e02-4599-b486-d492e9adb965', 'c7464789-5120-4b39-94d3-d950f174017c', '7fe407d0-1947-4dbf-861a-0b49b5fd5bce']


# Verify the Human Loops are Completed

In [10]:
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")
    while resp["HumanLoopStatus"] != "Completed":
        print(f"Waiting for HumanLoop to complete.")
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f"Completed!")
        print("")

HumanLoop Name: b30ce452-0e02-4599-b486-d492e9adb965
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2/2023/03/27/15/22/44/b30ce452-0e02-4599-b486-d492e9adb965/output.json'}

Completed!

HumanLoop Name: c7464789-5120-4b39-94d3-d950f174017c
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2/2023/03/27/15/22/44/c7464789-5120-4b39-94d3-d950f174017c/output.json'}

Completed!

HumanLoop Name: 7fe407d0-1947-4dbf-861a-0b49b5fd5bce
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2/2023/03/27/15/22/44/7fe407d0-1947-4dbf-861a-0b49b5fd5bce/o

# View Human Labels  

Once the work is complete, Amazon GroundTruth stores the results in the specified S3 bucket and sends a Cloudwatch Event.  Here is a sample item labeled with GroundTruth in `jsonlines` format:
```
{
 "inputContent": {"taskObject": {
                         "prompt": "Sometimes it works but usually not",
                         "responses": [2, 3]}
                 },
 "humanAnswers": [{"answerContent": {
                        "ranking_1": "1", # ranking for 1st response (1 is High)
                        "ranking_2": "2"  # ranking for 2nd response (2 is Low)
                 }}]
}
```

# Prepare human-labeled data for RL/PPO training
Retrieve from GrountTruth and convert to a binary reward (-1, 1) for all rankings as follows:

From this:
```
prompt                                response    ranking

Sometimes it works but usually not    2           1   # High
Sometimes it works but usually not    3           2   # Low
```

To this:
```
prompt                                response    ranking

Sometimes it works but usually not    2           1   # Ranked highest
Sometimes it works but usually not    3           0   # Not ranked highest
```

To this (`turn_into_text_classification_format()` below):
```
prompt                                response    highest_ranked_response_index

Sometimes it works but usually not    [2,3]       0   # 0th item in the response list is ranked the highest

```

# _Note:  If nothing is showing up below, you need to return to finish the previous notebook by labeling the data in Ground Truth!!_

In [11]:
import re
from pprint import pprint

human_feedback_items = []

for resp in completed_human_loops:
    human_feedback_s3_uri = resp["HumanLoopOutput"]["OutputS3Uri"]
    split_string = re.split("s3://" + bucket + "/", resp["HumanLoopOutput"]["OutputS3Uri"])
    key = split_string[1]
    
    response = s3.get_object(Bucket=bucket, Key=key)
    content = response["Body"].read().decode("utf-8")
    json_output = json.loads(content)
    print(json_output)

    prompt = json_output["inputContent"]['taskObject']['prompt']
    responses = json_output["inputContent"]['taskObject']['responses']
    response_1_ranking = json_output["humanAnswers"][0]["answerContent"]['response_1_ranking']
    response_2_ranking = json_output["humanAnswers"][0]["answerContent"]['response_2_ranking']
    
    human_feedback_item_1 = (prompt, responses[0], response_1_ranking)
    human_feedback_items.append(human_feedback_item_1)
    human_feedback_item_2 = (prompt, responses[1], response_2_ranking)
    human_feedback_items.append(human_feedback_item_2)

{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:079002598131:flow-definition/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2', 'humanAnswers': [{'acceptanceTime': '2023-03-27T15:23:29.605Z', 'answerContent': {'response_1_ranking': '2', 'response_2_ranking': '1'}, 'submissionTime': '2023-03-27T15:23:41.274Z', 'timeSpentInSeconds': 11.669, 'workerId': 'e7232bf5ab67e176', 'workerMetadata': {'identityData': {'identityProviderType': 'Cognito', 'issuer': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_WEvHYVrSh', 'sub': '06e39925-66bd-45b1-b495-9081ad730e85'}}}], 'humanLoopName': 'b30ce452-0e02-4599-b486-d492e9adb965', 'inputContent': {'taskObject': {'prompt': 'Given the following review:\nAwesome\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5', 'responses': [4, 5]}}}
{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:079002598131:flow-definition/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a314

In [12]:
df_human_feedback_items = pd.DataFrame(human_feedback_items, columns=['prompt', 'response', 'ranking'])
df_human_feedback_items.head(10)

,prompt,response,ranking
0,Given the following review:\nAwesome\npredict ...,4,2
1,Given the following review:\nAwesome\npredict ...,5,1
2,Given the following review:\nI keep buying mad...,2,1
3,Given the following review:\nI keep buying mad...,3,2
4,Given the following review:\nIf you are preppi...,4,2
5,Given the following review:\nIf you are preppi...,5,1


# Convert ranking into 0 or 1 reward

In [13]:
num_rankings = 2
df_human_feedback_items['response'] = df_human_feedback_items['response'].apply(lambda response: str(response))
df_human_feedback_items['ranking'] = df_human_feedback_items['ranking'].apply(lambda ranking: str(abs(int(ranking) - num_rankings)))
df_human_feedback_items.head(10)

,prompt,response,ranking
0,Given the following review:\nAwesome\npredict ...,4,0
1,Given the following review:\nAwesome\npredict ...,5,1
2,Given the following review:\nI keep buying mad...,2,1
3,Given the following review:\nI keep buying mad...,3,0
4,Given the following review:\nIf you are preppi...,4,0
5,Given the following review:\nIf you are preppi...,5,1


In [14]:
df_human_feedback_items_grouped_by_prompt = df_human_feedback_items.groupby('prompt', as_index=False).agg({'prompt' : 'first', 'response' : ','.join, 'ranking' : ','.join})
df_human_feedback_items_grouped_by_prompt



,prompt,response,ranking
0,Given the following review:\nAwesome\npredict ...,"4,5","0,1"
1,Given the following review:\nI keep buying mad...,"2,3","1,0"
2,Given the following review:\nIf you are preppi...,"4,5","0,1"


In [15]:
df_human_feedback_items_grouped_by_prompt['response'] = df_human_feedback_items_grouped_by_prompt['response'].apply(lambda response: [int(s) for s in response.split(',')])
df_human_feedback_items_grouped_by_prompt['ranking'] = df_human_feedback_items_grouped_by_prompt['ranking'].apply(lambda ranking: [int(s) for s in ranking.split(',')])
df_human_feedback_items_grouped_by_prompt



,prompt,response,ranking
0,Given the following review:\nAwesome\npredict ...,"[4, 5]","[0, 1]"
1,Given the following review:\nI keep buying mad...,"[2, 3]","[1, 0]"
2,Given the following review:\nIf you are preppi...,"[4, 5]","[0, 1]"


In [16]:
from datasets import Dataset

# Create Dataset objects (Arrow PyTables) from Pandas dataframes
human_feedback_dataset = Dataset.from_pandas(df_human_feedback_items_grouped_by_prompt)
human_feedback_dataset

Dataset({
    features: ['prompt', 'response', 'ranking'],
    num_rows: 3
})

# Train a reward model with human preference and alignment data
This is typically a language model initialized from the supervised-fine-tuned (SFT) model (trained in a previous notebook), but with an additional binary-classification layer placed on top.  This reward model is used to train the reinforcement-learning model in the next step.  The reinforcement-learning model is what is deployed into production to serve applications.

# TODO:  This should be bloomz (not bloom) at this point in the flow

In [17]:
%store -r model_checkpoint

In [18]:
model_checkpoint = 'bigscience/bloomz-560m'

In [19]:
try:
    model_checkpoint
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous section before you continue.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [20]:
print(model_checkpoint)

bigscience/bloomz-560m


In [21]:
%store -r supervised_fine_tuned_model_path

In [22]:
try:
    supervised_fine_tuned_model_path
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous section before you continue.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [23]:
print(supervised_fine_tuned_model_path)

./tmp_models/bigscience/bloom-560m/


In [24]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch.nn as nn
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

In [25]:
# Simulated (cooking show) output from SFT (Step 1) for Amazon Customer Review generative-classification task
#model_name = "bigscience/bloomz-560m" 

# Load the human comparisons dataset for tuning the reward model.
# Simulated output from HF Step 2 to be used to train reward for "helpfulness"
#ds = load_dataset("openai/summarize_from_feedback", name="comparisons") 

# Load the value-head model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [26]:
# Turn the dataset into pairs of prompt + responses, where text_j is the preferred prompt + response and text_k is the other.
def turn_into_text_classification_format(examples):
    new_examples = {"text_j": [], "text_k": []}
    for prompt, response, ranking in zip(examples["prompt"], examples["response"], examples["ranking"]):
        # TODO:  Add a check to make sure there is only a single 0 and a single 1
        if len(response) != 2 or len(ranking) != 2 or ranking[0] not in (0, 1) or ranking[1] not in (0, 1):
            raise ValueError(
                f"There should be two responses with a ranking that is either 0 or 1. Received {len(response)} responses and {len(ranking)} rankings."
            )
            
        highest_ranked_response_index = ranking.index(1)

        new_examples["text_j"].append(
            #str(response[highest_ranked_response_index]) + " " + tokenizer.bos_token + " " + prompt
            prompt + " " + str(response[highest_ranked_response_index])
        )
        new_examples["text_k"].append(
            #str(response[0 if highest_ranked_response_index == 1 else 1]) + " " + tokenizer.bos_token + " " + prompt
            prompt + " " + str(response[0 if highest_ranked_response_index == 1 else 1])
        )

    return new_examples

# Tokenize the dataset.
def preprocess_function(examples):
    tokenized_j = tokenizer(examples["text_j"], truncation=True)
    tokenized_k = tokenizer(examples["text_k"], truncation=True)
    return {
        "input_ids_j": tokenized_j["input_ids"],
        "attention_mask_j": tokenized_j["attention_mask"],
        "input_ids_k": tokenized_k["input_ids"],
        "attention_mask_k": tokenized_k["attention_mask"],
    }


In [27]:
num_proc = 8  # Can adjust to be higher if you have more processors. Should work even if you don't have 8 CPUs, though.
original_columns = human_feedback_dataset.column_names
print(original_columns)

human_feedback_binary_classification_dataset = human_feedback_dataset.map(turn_into_text_classification_format, batched=True, num_proc=num_proc, remove_columns=original_columns)

human_feedback_tokenized_dataset = human_feedback_binary_classification_dataset.map(preprocess_function, 
                                                                                    batched=True, 
                                                                                    num_proc=num_proc, 
                                                                                    remove_columns=["text_j", "text_k"])

print(human_feedback_tokenized_dataset)


num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


['prompt', 'response', 'ranking']


#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2: 100%|██████████| 1/1 [00:00<00:00, 269.78ba/s]
num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.

#1:   0%|          | 0/1 [00:00<?, ?ba/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


#0: 100%|██████████| 1/1 [00:00<00:00, 155.89ba/s]


Dataset({
    features: ['input_ids_j', 'attention_mask_j', 'input_ids_k', 'attention_mask_k'],
    num_rows: 3
})


In [28]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1)

# # Need to do this for gpt2, because it doesn't have an official pad token.
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = tokenizer.eos_token_id

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloomz-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Define the metric that we'll use for validation.
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    # Here, predictions is rewards_j and rewards_k.
    # We want to see how much of the time rewards_j > rewards_k.
    predictions = np.argmax(predictions, axis=0)
    labels = np.zeros(predictions.shape)
    return accuracy.compute(predictions=predictions, references=labels)

In [30]:
# We need to define a special data collator that batches the data in our j vs k format.
@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append({"input_ids": feature["input_ids_j"], "attention_mask": feature["attention_mask_j"]})
            features_k.append({"input_ids": feature["input_ids_k"], "attention_mask": feature["attention_mask_k"]})
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }
        return batch

In [31]:
class RewardTrainer(Trainer):
    # Define how to compute the reward loss.
    def compute_loss(self, model, inputs, return_outputs=False):
        rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
        if return_outputs:
            return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
        return loss

# Define and parse arguments.
local_rank = 0
resume_from_checkpoint = False
deepspeed = None
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
gradient_accumulation_steps = 4
learning_rate = 2e-5
weight_decay = 0.001
bf16 = False
num_train_epochs = 1

# Define the training args. Needs to be done before the model is loaded if you are using deepspeed.
training_args = TrainingArguments(
    output_dir=f"{model_checkpoint.replace('/', '_')}_reward_model",
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
#    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=gradient_accumulation_steps,
#    deepspeed=deepspeed,
#    local_rank=local_rank,
    remove_unused_columns=False,
    label_names=[],
)
    
# Train the model, woohoo.
trainer = RewardTrainer(
    model=model,
    args=training_args,
    train_dataset=human_feedback_tokenized_dataset, #["train"],
#    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer),
)

trainer.train(resume_from_checkpoint)

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 4
  Total optimization steps = 1
  Number of trainable parameters = 559215616


[2023-03-27 15:24:27.103: W smdistributed/modelparallel/torch/nn/predefined_hooks.py:78] Found unsupported HuggingFace version 4.26.1 for automated tensor parallelism. HuggingFace modules will not be automatically distributed. You can use smp.tp_register_with_module API to register desired modules for tensor parallelism, or directly instantiate an smp.nn.DistributedModule. Supported HuggingFace transformers versions for automated tensor parallelism: ['4.17.0', '4.20.1', '4.21.0']
[2023-03-27 15:24:27.139 pytorch-1-12-gpu--ml-p3dn-24xlarge-307ebad80d11874f5dcc2ce687db:9482 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-03-27 15:24:27.264 pytorch-1-12-gpu--ml-p3dn-24xlarge-307ebad80d11874f5dcc2ce687db:9482 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


/opt/conda/lib/python3.8/site-packages/smdebug-1.0.24b20230214-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.24b20230214-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


Saving model checkpoint to bigscience_bloomz-560m_reward_model/checkpoint-1
Configuration saved in bigscience_bloomz-560m_reward_model/checkpoint-1/config.json
Model weights saved in bigscience_bloomz-560m_reward_model/checkpoint-1/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1, training_loss=0.22319945693016052, metrics={'train_runtime': 60.2252, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.017, 'total_flos': 0.0, 'train_loss': 0.22319945693016052, 'epoch': 1.0})

In [32]:
reward_model_path = './tmp_models/reward_model/'

trainer.save_model(reward_model_path)

Saving model checkpoint to ./tmp_models/reward_model/
Configuration saved in ./tmp_models/reward_model/config.json
Model weights saved in ./tmp_models/reward_model/pytorch_model.bin


In [33]:
%store reward_model_path

Stored 'reward_model_path' (str)


In [34]:
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_path, num_labels=1)

loading configuration file ./tmp_models/reward_model/config.json
Model config BloomConfig {
  "_name_or_path": "./tmp_models/reward_model/",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}

loadin

In [42]:
from transformers import TextClassificationPipeline
from transformers import pipeline

#device = torch.device('cuda' if use_cuda else 'cpu')

pipeline_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

reward_model_pipeline = pipeline("text-classification", tokenizer=pipeline_tokenizer, model=reward_model)

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloomz-560m/snapshots/25f241f41c04f08d658a1dd3b49ad41390109a8e/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloomz-560m/snapshots/25f241f41c04f08d658a1dd3b49ad41390109a8e/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bigscience--bloomz-560m/snapshots/25f241f41c04f08d658a1dd3b49ad41390109a8e/tokenizer_config.json


In [70]:
prompt = 'Given the following review:\nAwesome\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5'
response = 3
reward_model_pipeline.predict(prompt + ' ' + str(response))

[{'label': 'LABEL_0', 'score': 1.0210785603703698e-06}]

In [65]:
prompt = 'Given the following review:\nIf you are prepping for the end of the world this is one of those things that you should have installed on your-end-of-the-world-proof PC.  Hail to the great Yuri!\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5'
response = 4
reward_model_pipeline.predict(prompt + ' ' + str(response))

[{'label': 'LABEL_0', 'score': 0.004329970572143793}]

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>